In [1]:
import os
import pandas as pd
import numpy as np
import branca
import folium

from folium.plugins import TimestampedGeoJson
from geopy.geocoders import Nominatim
from pywindsorai.client import Client

API_KEY = os.environ["WINDSOR_TOKEN"]
geolocator = Nominatim(user_agent="Worldmap for Google Ads Clicks")

In [2]:
client = Client(API_KEY)
dataset = client.connectors(
    date_from="2022-10-01",
    date_to="2022-11-01",
    fields=["date", "country", "source", "campaign", "clicks"],
    connector="google_ads"
)
dataset = pd.DataFrame(dataset["data"])

In [3]:
print(len(dataset))
print(len(dataset["country"].unique()))
print(dataset["clicks"].max())
print(dataset["clicks"].min())
dataset.head()

3388
176
1121
1


,date,country,source,campaign,clicks
0,2022-10-08,Afghanistan,google,pmax test,1
1,2022-10-08,Afghanistan,google,[MB] Sales | PerfMax | WW,5
2,2022-10-10,Afghanistan,google,pmax test,11
3,2022-10-11,Afghanistan,google,pmax test,6
4,2022-10-11,Albania,google,pmax test,2


In [4]:
lat, long = [], []
unique_countries = dataset["country"].unique()
for c in unique_countries:
    location = geolocator.geocode(c)
    print(location)
    if location:
        lat.append(location.latitude)
        long.append(location.longitude)
    else:
        lat.append(float("Nan"))
        long.append(float("Nan"))

افغانستان
Shqipëria
Algérie / ⵍⵣⵣⴰⵢⴻⵔ / الجزائر
Angola
Antigua and Barbuda
Azərbaycan
Argentina
Australia
Österreich
البحرين
বাংলাদেশ
Հայաստան
Barbados
België / Belgique / Belgien
འབྲུགཡུལ་
Bolivia
Bosna i Hercegovina / Босна и Херцеговина
Botswana
Brasil
Belize
Brunei
България
မြန်မာ
Burundi
Беларусь
ព្រះរាជាណាចក្រ​កម្ពុជា
Cameroun
Canada
Cabo Verde
Cayman Islands
ශ්‍රී ලංකාව இலங்கை
Tchad تشاد
中国
臺灣
Colombia
Congo
République démocratique du Congo
Costa Rica
Hrvatska
Hévízgyörk, Aszódi járás, Pest megye, Közép-Magyarország, 2192, Magyarország
Česko
Bénin
Danmark
República Dominicana
República Dominicana
Ecuador
El Salvador
ኢትዮጵያ
Eesti
Viti
Suomi / Finland
France
Guyane, France
Gabon
Georgia, United States
Gambia
الأراضي الفلسطينية, Palestinian Territory
Deutschland
Ghana
Ελλάς
Grenada
Guadeloupe, France
Guam, Chalan Pago-Ordot Municipality, Guam, United States
Guatemala
Guinée
Guyana
Ayiti
Honduras
香港島 Hong Kong Island, 香港 Hong Kong, 中国
Magyarország
Ísland
India
Indonesia
Ócsa, Gyáli j

In [5]:
lat_long_data = pd.DataFrame(
    data = {
        "country": list(unique_countries),
        "latitude": lat,
        "longitude": long
    }
)

In [6]:
dataset = dataset.merge(lat_long_data, on="country", how="left")

In [7]:
dataset

,date,country,source,campaign,clicks,latitude,longitude
0,2022-10-08,Afghanistan,google,pmax test,1,33.768006,66.238514
1,2022-10-08,Afghanistan,google,[MB] Sales | PerfMax | WW,5,33.768006,66.238514
2,2022-10-10,Afghanistan,google,pmax test,11,33.768006,66.238514
3,2022-10-11,Afghanistan,google,pmax test,6,33.768006,66.238514
4,2022-10-11,Albania,google,pmax test,2,41.000028,19.999962
...,...,...,...,...,...,...,...
3383,2022-10-28,Kosovo,google,pmax test,2,42.586958,20.902123
3384,2022-10-29,Kosovo,google,pmax test,5,42.586958,20.902123
3385,2022-10-30,Kosovo,google,pmax test,7,42.586958,20.902123
3386,2022-10-31,Kosovo,google,pmax test,1,42.586958,20.902123


In [8]:
# dataset["clicks_normalized"] = (dataset["clicks"] - dataset["clicks"].mean()) / dataset["clicks"].std()
# dataset["clicks_normalized"] = dataset["clicks_normalized"].apply(lambda x: x if x > 1 else 1)
# dataset.sort_values('clicks_normalized', ascending=False).head()
# dataset["clicks"] = np.random.randint(1, 10, size=len(dataset))
dataset["clicks"] = np.random.normal(1, 5, size=len(dataset))
dataset["clicks"] = dataset["clicks"].apply(lambda x: 1 if x <= 1 else x)
dataset

,date,country,source,campaign,clicks,latitude,longitude
0,2022-10-08,Afghanistan,google,pmax test,1.000000,33.768006,66.238514
1,2022-10-08,Afghanistan,google,[MB] Sales | PerfMax | WW,5.406883,33.768006,66.238514
2,2022-10-10,Afghanistan,google,pmax test,7.627240,33.768006,66.238514
3,2022-10-11,Afghanistan,google,pmax test,7.753712,33.768006,66.238514
4,2022-10-11,Albania,google,pmax test,1.962647,41.000028,19.999962
...,...,...,...,...,...,...,...
3383,2022-10-28,Kosovo,google,pmax test,5.287743,42.586958,20.902123
3384,2022-10-29,Kosovo,google,pmax test,1.000000,42.586958,20.902123
3385,2022-10-30,Kosovo,google,pmax test,1.000000,42.586958,20.902123
3386,2022-10-31,Kosovo,google,pmax test,1.000000,42.586958,20.902123


In [9]:
colormap = branca.colormap.LinearColormap(
    colors=["#b35549", "#a6ac24", "#1da976", "#1596ce"],
    vmin=dataset["clicks"].min(), 
    vmax=dataset["clicks"].max(),
    caption = 'Google Ads Clicks By Country (October 2022)'
)

def create_geojson_features(df):
    features = []
    
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[row['longitude'],row['latitude']]
            },
            'properties': {
                'time': pd.to_datetime(row['date']).__str__(),
                'style': {'color' : ''},
                'icon': 'circle',
                'tooltip': row["country"],
                'iconstyle':{
                    'fillColor': colormap(row["clicks"]),
                    'fillOpacity': 0.4,
                    'stroke': 'true',
                    'radius': row['clicks']
                }
            }
        }
        features.append(feature)
    return features

In [10]:
geojson = create_geojson_features(dataset)

In [11]:
geojson[0]

{'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [66.2385139, 33.7680065]},
 'properties': {'time': '2022-10-08 00:00:00',
  'style': {'color': ''},
  'icon': 'circle',
  'tooltip': 'Afghanistan',
  'iconstyle': {'fillColor': '#b35549ff',
   'fillOpacity': 0.4,
   'stroke': 'true',
   'radius': 1.0}}}

In [12]:
world_map = folium.Map(
    tiles="https://{s}.basemaps.cartocdn.com/dark_nolabels/{z}/{x}/{y}{r}.png",
    attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>',
    zoom_control=False,
    scrollWheelZoom=False,
    dragging=False,
    doubleClickZoom=False,
    attributionControl=False,
    location=[0, 30],
    zoom_start=1.5,
)

TimestampedGeoJson(
    geojson,
    period = 'P1D',
    duration = 'P1D',
    transition_time = 1000,
    auto_play = True,

).add_to(world_map)

html_to_insert = """
    <style>
        .caption {
            fill: white !important; 
        }
        
        .tick {
            fill: white !important;
        }
        
        .leaflet-bar {
            display: none !important;
        }
        
    </style>
"""
world_map.get_root().header.add_child(folium.Element(html_to_insert))
colormap.add_to(world_map)
world_map

In [13]:
world_map.save("test.html")

TODO:
* add random data to give more action, (done)
* add initial position closer in map initialization (done)
* save as gif, (done)
* remove play buttons (done)
* remove zoom, (done)
* Format code,
* Save code as .py and notebook
* make sure to add dependencies for selenium firefox, chrome.


In [14]:
from selenium import webdriver
import time
driver_option = "firefox"
if driver_option == "chrome":
    options = webdriver.ChromeOptions()
    options.add_argument('window-size=1024x768')
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
elif driver_option == "firefox":
    options = webdriver.FirefoxOptions()
    options.add_argument('window-size=1024x768')
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options)

file_path = f"file://{os.getcwd()}/test.html"
driver.get(file_path)
for i in range(30):
    time.sleep(1)
    driver.save_screenshot(f"test_img/img{i}.png")
driver.quit()

In [16]:
from PIL import Image
from pathlib import Path

all_images = []
for path in Path('test_img').rglob('*.png'):
    all_images.append(Image.open(path))

all_images[0].save(
    "world_map.gif",
    format="GIF",
    append_images=all_images[1:],
    save_all=True,
    duration=1000,
    loop=100
)

In [19]:
options = getattr(webdriver, "ChromeOptions")()
options.add_argument('window-size=1024x768')
options.add_argument("--headless")

In [22]:
getattr(webdriver, "Firefox")(options=options)

AttributeError: 'Options' object has no attribute 'binary'